In [43]:
# -*- coding: utf-8 -*-
from sklearn import svm
import pandas as pd
import numpy as np
import sys
import csv

params = {}
params['nu'] = 0.001
params['gamma']='auto'
params['kernel'] ='poly'
params['path'] = "methodfea_train01new.csv"
params['opath'] ='data1_ocs.csv'
argvs = sys.argv
try:
    with open(params['path'],'r') as f:
    #1.创建阅读器对象
        reader = csv.reader(f)
    #2.读取文件第一行数据
        head_row=next(reader)
    data_attribute = []
    for item in head_row:
        data_attribute.append(item)

#读取数据并删除最后一列标签
    tn = pd.read_csv(params['path']) 
    tn.dropna(inplace=True)
    train = np.array(tn)
    train_x = train[:, :-1]
#存标签
    train_y = train[:,-1]
    train_y = np.array(train_y)

#对所有数据行进行异常检测
    train_x = np.array(train_x)
    clf = svm.OneClassSVM(nu=params['nu'],
              kernel=params['kernel'],
              gamma=params['gamma']).fit(train_x)
#pred存入的是每一行数据的预测值，是1或者-1
    pred = clf.predict(train_x)
    normal = train_x[pred == 1]
    abnormal = train_x[pred == -1]
#删除pred为-1的行数据
    df = pd.DataFrame(pd.read_csv(params['path']))[0:pred.size]
    df['pred']=pred
    df2 = df[-df.pred.isin([-1])]
    df2 = df2.drop(['pred'],axis=1)
#将清洗之后的数据存入csv文件
    data_out = df2.iloc[:,:].values
    csvfile2 = open(params['opath'],'w')
    writer = csv.writer(csvfile2)
    writer.writerow(data_attribute)   #存属性
    m=len(data_out)
    for i in range(m):
        writer.writerow(data_out[i])
except Exception as e:
    print(e)

In [55]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy import signal
from librosa import feature
import statsmodels.api as sm
from itertools import chain
from pywt import wavedec
import python_speech_features


def time_feaget(root_file, name_file):
    df = pd.read_csv(root_file)
    result_list = []
    for i in df.columns:
        flist, plist = signal.welch(df[i], 25600)
        main_ener1 = np.square(plist[np.logical_and(flist >= 1600,
                                                    flist <    2400)]).sum()
        main_ener2 = np.square(plist[np.logical_and(flist >= 3600,
                                                    flist < 3950)]).sum()

        # 输出均值 标准差 方差 偏度 峰度 极差
        list_para = [df[i].mean(), df[i].std(),np.var(df[i]),
        stats.skew(df[i]),stats.kurtosis(df[i]), df[i].ptp(), ]
        #print(len(ar_result.params))
        list_para.extend(ar_result.params)
        result_list.extend(list_para)
    return result_list
def freq_feaget(root_file, name_file):
    # 输出频带能量比
    flist, plist = signal.welch(df[i], 25600)
    main_ener1 = np.square().sum()
    main_ener2 = np.square().sum()
    list_param = [df[i].ptp(), ]
    #print(len(ar_result.params))
    list_para.extend(ar_result.params)
    result_list.extend(list_para)
    return result_list
def timefreq_feaget(root_file, name_file):
    # 输出cA5 cD5 cD4 cD3 cD2 cD1能量
    cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(df[i], 'db10',level=5)
    ener_cA5 = np.square(cA5).sum()
    ener_cD5 = np.square(cD5).sum()
    ener_cD4 = np.square(cD4).sum()
    ener_cD3 = np.square(cD3).sum()
    ener_cD2 = np.square(cD2).sum()
    ener_cD1 = np.square(cD1).sum()
    list_para = [ener_cA5, ener_cD5,ener_cD4,ener_cD3,ener_cD2,ener_cD1]
    list_para.extend(ar_result.params)
    result_list.extend(list_para)
    return result_list

In [56]:
t=time_feaget(params['opath'],1)

AttributeError: 'Series' object has no attribute 'ptp'

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1_scaler, Y1, test_size=0.3, random_state=123)#以刀具1为训练集划分训练和验证集
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# 将参数写成字典下形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression', # 目标函数
    #'metric': {'l2', 'auc'},  # 评估函数
    'metric': 'rmse',  # 评估函数
    'num_leaves': 126,   # 叶子节点数
    'learning_rate': 0.05,  # 学习速率
    'feature_fraction': 0.9, # 建树的特征选择比例
    'bagging_fraction': 0.8, # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
gbm = lgb.train(params,lgb_train,num_boost_round=500,valid_sets=[lgb_train,lgb_eval],early_stopping_rounds=10)
with open("methodfea_train03new.csv") as f:
    reader = csv.reader(f)
    head_row=next(reader)
    data_attribute = []
    for item in head_row:
        data_attribute.append(item)
    tn = pd.read_csv("methodfea_train03new.csv") 
    tn.dropna(inplace=True)
    train = np.array(tn)
    train_x = train[:, :-1]
    train_x = np.array(train_x)
    train_y = train[:,-1]
    train_y = np.array(train_y)
X3_scaler= train_x
# print(X3_scaler)
yp = gbm.predict(X3_scaler, num_iteration=gbm.best_iteration)   #预测值
yt =train_y     #真实值

[1]	training's rmse: 3853.35	valid_1's rmse: 3841.58
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 3660.78	valid_1's rmse: 3649.72
[3]	training's rmse: 3477.81	valid_1's rmse: 3467.42
[4]	training's rmse: 3304	valid_1's rmse: 3294.23
[5]	training's rmse: 3138.9	valid_1's rmse: 3129.73
[6]	training's rmse: 2982.02	valid_1's rmse: 2973.32
[7]	training's rmse: 2832.99	valid_1's rmse: 2824.79
[8]	training's rmse: 2691.41	valid_1's rmse: 2683.64
[9]	training's rmse: 2556.94	valid_1's rmse: 2549.6
[10]	training's rmse: 2429.17	valid_1's rmse: 2422.33
[11]	training's rmse: 2307.77	valid_1's rmse: 2301.33
[12]	training's rmse: 2192.45	valid_1's rmse: 2186.44
[13]	training's rmse: 2087.93	valid_1's rmse: 2083.03
[14]	training's rmse: 1983.67	valid_1's rmse: 1979.11
[15]	training's rmse: 1884.6	valid_1's rmse: 1880.42
[16]	training's rmse: 1799.66	valid_1's rmse: 1798.95
[17]	training's rmse: 1709.77	valid_1's rmse: 1709.46
[18]	training's rmse: 1624.39	valid_

[154]	training's rmse: 24.7562	valid_1's rmse: 81.0207
[155]	training's rmse: 24.5825	valid_1's rmse: 81.015
[156]	training's rmse: 24.4341	valid_1's rmse: 80.9951
[157]	training's rmse: 24.2992	valid_1's rmse: 80.971
[158]	training's rmse: 24.1704	valid_1's rmse: 80.9544
[159]	training's rmse: 24.0441	valid_1's rmse: 80.9577
[160]	training's rmse: 23.9236	valid_1's rmse: 80.9447
[161]	training's rmse: 23.7331	valid_1's rmse: 80.9689
[162]	training's rmse: 23.5834	valid_1's rmse: 80.939
[163]	training's rmse: 23.4288	valid_1's rmse: 80.9277
[164]	training's rmse: 23.2967	valid_1's rmse: 80.911
[165]	training's rmse: 23.1226	valid_1's rmse: 80.8894
[166]	training's rmse: 22.9611	valid_1's rmse: 80.8673
[167]	training's rmse: 22.8377	valid_1's rmse: 80.8676
[168]	training's rmse: 22.6871	valid_1's rmse: 80.9045
[169]	training's rmse: 22.5571	valid_1's rmse: 80.9434
[170]	training's rmse: 22.4358	valid_1's rmse: 80.9664
[171]	training's rmse: 22.3076	valid_1's rmse: 80.9756
[172]	training

In [37]:
from sklearn.metrics import mean_squared_error #均方误差
print("RMSE is:")
print(np.sqrt(mean_squared_error(yt/60,yp/60)))

#评价分数
def scores(y_true, y_pred):
    er = (y_true - y_pred)/60
    def apply_each(x):
        if(x <= 0):
            return np.exp(-np.log(0.5) * (x / 5))
        else:
            return np.exp(np.log(0.5) * (x / 20))
    return 'score', np.array([apply_each(i) for i in er]).mean()*100, True
print("分数 is:")
print(scores(yt,yp))

RMSE is:
55.802712329594684
分数 is:
('score', 0.05371089509447023, True)
